In [1]:
import os

from google.colab import drive
Drive = drive.mount('/content/drive')

print("Connected")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Connected


In [3]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
import numpy as np
import scipy.io
from scipy import signal
from scipy.stats import pearsonr
from scipy.interpolate import CubicSpline
from scipy.ndimage import gaussian_filter1d
from tqdm import tqdm

# Check GPU availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

Using device: cuda


In [ ]:
def augment_data(ecog, glove, noise_std=0.1, time_shift=5):
    # Add Gaussian noise
    noisy_ecog = ecog + np.random.normal(0, noise_std, ecog.shape)

    # Time shift
    shift = np.random.randint(-time_shift, time_shift)
    if shift > 0:
        shifted_ecog = np.vstack([ecog[:shift], ecog[:-shift]])
        shifted_glove = np.vstack([glove[:shift], glove[:-shift]])
    elif shift < 0:
        shifted_ecog = np.vstack([ecog[-shift:], ecog[:shift]]).copy()  # Create a copy here
        shifted_glove = np.vstack([glove[-shift:], glove[:shift]]).copy()  # Create a copy here
    else:
        shifted_ecog = ecog.copy()
        shifted_glove = glove.copy()

    return noisy_ecog, shifted_ecog, shifted_glove

In [ ]:
class EnsembleModel:
    def __init__(self, n_models=5):
        self.models = [HybridECoGModel() for _ in range(n_models)]

    def train(self, ecog, glove):
        for i, model in enumerate(self.models):
            print(f"Training model {i+1}/{len(self.models)}")
            model.train(ecog, glove)

    def predict(self, ecog):
        predictions = []
        for model in self.models:
            pred = model.predict(ecog)
            predictions.append(pred)
        return np.mean(predictions, axis=0)

In [ ]:
# ========== Transformer Model Definition ==========
class TransformerModel(nn.Module):
    def __init__(self, input_dim, output_dim, d_model=256, nhead=8, num_layers=3, dropout=0.1):
        super().__init__()
        self.encoder = nn.Linear(input_dim, d_model)
        self.pos_encoder = PositionalEncoding(d_model, dropout)
        encoder_layers = nn.TransformerEncoderLayer(d_model, nhead, d_model*4, dropout)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        self.decoder = nn.Linear(d_model, output_dim)

    def forward(self, src):
        src = self.encoder(src) * np.sqrt(src.shape[-1])
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src)
        output = self.decoder(output)
        return output

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super().__init__()



class PositionalEncoding(nn.Module):
    def __init__(self, d_model, dropout=0.1, max_len=5000):
        super().__init__()
        self.dropout = nn.Dropout(p=dropout)
        position = torch.arange(max_len).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2) * (-np.log(10000.0) / d_model))
        pe = torch.zeros(max_len, d_model)
        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0), :]
        return self.dropout(x)

In [ ]:
# ========== Gaussian Smoothing (All points) ==========
def gaussian_smoothing(pred, sigma=2.25):
    pred_smooth = pred.copy()
    for i in range(pred.shape[1]):
        pred_smooth[:, i] = gaussian_filter1d(pred[:, i], sigma=sigma)
    return pred_smooth

# ========== Outlier Suppression (Set to 0) ==========
def suppress_low_outliers(pred, threshold_multiplier=2):
    pred_cleaned = pred.copy()
    for i in range(pred.shape[1]):
        col = pred[:, i]
        mean = np.mean(col)
        std = np.std(col)
        threshold = mean - threshold_multiplier * std
        pred_cleaned[:, i] = np.where(col < threshold, 0, col)
    return pred_cleaned

# ========== Bandpass Filter ==========
def bandpass_filter(data, fs, lowcut, highcut, order=4):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = signal.butter(order, [low, high], btype='band')
    return signal.filtfilt(b, a, data, axis=0)

# ========== Feature Extraction ==========
def get_features(window, fs=1000):
    freq_bands = [(5, 15), (20, 25), (75, 115), (125, 160), (160, 175)]
    n_channels = window.shape[1]
    features = np.zeros((n_channels, 6))
    features[:, 0] = np.mean(window, axis=0)
    for i, (low, high) in enumerate(freq_bands):
        band_filtered = bandpass_filter(window, fs, low, high)
        features[:, i + 1] = np.mean(np.abs(band_filtered), axis=0)
    return features

# ========== Sliding Window ==========
def get_windowed_feats(ecog, fs, win_len, overlap):
    step = win_len - overlap
    num_windows = (ecog.shape[0] - overlap) // step
    feats = []
    for i in tqdm(range(num_windows), desc="Extracting features"):
        start = i * step
        end = start + win_len
        if end > ecog.shape[0]:
            break
        window = ecog[start:end, :]
        feats.append(get_features(window, fs).flatten())
    return np.array(feats)

# ========== Create R Matrix ==========
def create_R_matrix(features, N_wind):
    num_windows, num_feats = features.shape
    pad = np.tile(features[0], (N_wind - 1, 1))
    padded = np.vstack([pad, features])
    R = np.zeros((num_windows, 1 + N_wind * num_feats))
    for i in range(num_windows):
        context = padded[i:i + N_wind].flatten()
        R[i] = np.concatenate(([1], context))
    return R

In [ ]:
# ========== Hybrid Model ==========
class HybridECoGModel:
    def __init__(self, fs=1000, window_len=100, overlap=50, N_wind=3, n_splits=5):
        self.fs = fs
        self.window_len = window_len
        self.overlap = overlap
        self.step = window_len - overlap
        self.N_wind = N_wind
        self.n_splits = n_splits
        self.scaler = StandardScaler()
        self.model = None

    # Keep your existing feature extraction methods
    def bandpass_filter(self, data, lowcut, highcut, order=4):
        nyq = 0.5 * self.fs
        low = lowcut / nyq
        high = highcut / nyq
        b, a = signal.butter(order, [low, high], btype='band')
        return signal.filtfilt(b, a, data, axis=0)

    def get_features(self, window):
        freq_bands = [(5, 15), (20, 25), (75, 115), (125, 160), (160, 175)]
        n_channels = window.shape[1]
        features = np.zeros((n_channels, 6))
        features[:, 0] = np.mean(window, axis=0)
        for i, (low, high) in enumerate(freq_bands):
            band_filtered = self.bandpass_filter(window, low, high)
            features[:, i + 1] = np.mean(np.abs(band_filtered), axis=0)
        return features

    def get_windowed_feats(self, ecog):
        num_windows = (ecog.shape[0] - self.overlap) // self.step
        feats = []
        for i in range(num_windows):
            start = i * self.step
            end = start + self.window_len
            if end > ecog.shape[0]:
                break
            window = ecog[start:end, :]
            feats.append(self.get_features(window).flatten())
        return np.array(feats)

    def create_R_matrix(self, features):
        num_windows, num_feats = features.shape
        pad = np.tile(features[0], (self.N_wind - 1, 1))
        padded = np.vstack([pad, features])
        R = np.zeros((num_windows, 1 + self.N_wind * num_feats))
        for i in range(num_windows):
            context = padded[i:i + self.N_wind].flatten()
            R[i] = np.concatenate(([1], context))
        return R

    def train(self, ecog, glove):
        # Feature extraction
        feats = self.get_windowed_feats(ecog)
        R = self.create_R_matrix(feats)

        # Downsample glove data
        #glove_down = signal.decimate(glove, glove.shape[0] // R.shape[0], axis=0, zero_phase=True)[:R.shape[0]]
        glove_down = signal.decimate(glove, glove.shape[0] // R.shape[0], axis=0, zero_phase=True)[:R.shape[0]].copy() #copy here

        # Smooth glove data
        for i in range(glove_down.shape[1]):
            glove_down[:, i] = gaussian_filter1d(glove_down[:, i], sigma=2.25)

        # Prepare data for transformer
        X = R[:, 1:]  # Remove bias term
        y = glove_down

        # Normalize
        X = self.scaler.fit_transform(X)

        # Convert to PyTorch tensors
        X_tensor = torch.FloatTensor(X).unsqueeze(1).to(device)  # Add sequence dimension
        y_tensor = torch.FloatTensor(y).to(device)

        # Initialize transformer
        input_dim = X.shape[1]
        output_dim = y.shape[1]
        self.model = TransformerModel(input_dim, output_dim).to(device)

        # Training setup
        criterion = nn.MSELoss()
        optimizer = torch.optim.Adam(self.model.parameters(), lr=1e-4, weight_decay=1e-5)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5)

        # Create dataset
        dataset = torch.utils.data.TensorDataset(X_tensor, y_tensor)
        train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

        # Training loop
        n_epochs = 100
        best_loss = float('inf')

        for epoch in range(n_epochs):
            self.model.train()
            epoch_loss = 0
            for batch_X, batch_y in train_loader:
                optimizer.zero_grad()
                outputs = self.model(batch_X)
                loss = criterion(outputs.squeeze(1), batch_y)
                loss.backward()
                optimizer.step()
                epoch_loss += loss.item()

            epoch_loss /= len(train_loader)
            scheduler.step(epoch_loss)

            if epoch_loss < best_loss:
                best_loss = epoch_loss
                torch.save(self.model.state_dict(), 'best_model.pth')

            if (epoch + 1) % 10 == 0:
                print(f'Epoch {epoch+1}/{n_epochs}, Loss: {epoch_loss:.4f}')
    def predict(self, ecog, batch_size=32):  # Add batch_size argument
        if self.model is None:
            raise ValueError("Model not trained yet")

        # Load best model
        self.model.load_state_dict(torch.load('best_model.pth'))
        self.model.eval()

        # Feature extraction
        feats = self.get_windowed_feats(ecog)
        R = self.create_R_matrix(feats)
        X = R[:, 1:]
        X = self.scaler.transform(X)

        # Split data into batches
        num_batches = int(np.ceil(X.shape[0] / batch_size))
        predictions = []

        for i in range(num_batches):
            start = i * batch_size
            end = min(start + batch_size, X.shape[0])
            X_batch = X[start:end]

            # Convert to PyTorch tensors and add sequence dimension
            X_tensor = torch.FloatTensor(X_batch).unsqueeze(1).to(device)

            # Make predictions for the batch
            with torch.no_grad():
                batch_predictions = self.model(X_tensor)

            # Append batch predictions to the overall predictions
            # Ensure all tensors are squeezed to the same dimensions before appending:
            predictions.append(batch_predictions.squeeze(1).cpu().numpy())

        # Concatenate batch predictions and return
        # Explicitly pad the predictions to ensure they have the same shape before concatenation.
        max_len = max(pred.shape[0] if pred.ndim > 0 else 0 for pred in predictions) #Check if 1D, if so, default to 0 #FIXED: changed ndim > 1 to ndim > 0 and default to 0
        padded_predictions = []
        for pred in predictions:
            if pred.ndim == 2 and pred.size > 0:  # Check for 2D and non-empty
                pad_width = ((0, max_len - pred.shape[0]), (0, 0))
            elif pred.ndim == 1 and pred.size > 0: # Check for 1D and non-empty
                pad_width = (0, max_len - pred.shape[0])
            else:
            # Handle cases where pred is empty or has unexpected dimensions
            # You might want to raise an exception or fill with zeros
                pad_width = (0, max_len) if max_len > 0 else (0, 0)  # Pad to max_len or leave as is if max_len is 0

        padded_predictions.append(np.pad(pred, pad_width, 'constant'))

        return np.vstack(padded_predictions)

In [ ]:
# ========== Main Execution ==========
if __name__ == "__main__":
    # Load data
    train_data = scipy.io.loadmat('/content/drive/MyDrive/raw_training_data.mat')
    test_data = scipy.io.loadmat('/content/drive/MyDrive/leaderboard_data.mat')
    train_ecogs = train_data['train_ecog']
    train_gloves = train_data['train_dg']
    leaderboard_ecogs = test_data['leaderboard_ecog']

    # Initialize model
    model = HybridECoGModel()

    # Train and predict for each subject
    predicted_dg = np.empty((3, 1), dtype=object)

    for subj_idx in range(3):
        print(f"\n=== Processing Subject {subj_idx + 1} ===")
        ecog_train = train_ecogs[subj_idx].item()
        glove_train = train_gloves[subj_idx].item()
        ecog_test = leaderboard_ecogs[subj_idx].item()

        # Train model
        print("Training model...")
        model.train(ecog_train, glove_train)

        # Make predictions
        print("Making predictions...")
        #pred = model.predict(ecog_test)
        print("Making predictions...")
        pred = model.predict(ecog_test, batch_size=32)  # Specify batch size

        # Reshape predictions to 2D if necessary before padding
        if pred.ndim > 2:
            pred = pred.reshape(pred.shape[0], -1) # Reshape to 2D

        # Ensure predictions have the correct shape
        if pred.shape[0] < 147500:
            padding_size = 147500 - pred.shape[0]
            padding = np.zeros((padding_size, pred.shape[1]))
            pred = np.concatenate((pred, padding))
        elif pred.shape[0] > 147500:
            pred = pred[:147500, :]

        # Reshape to 147500 x 5 if necessary
        if pred.shape[1] != 5:
            pred = pred[:, :5]  # Truncate if more than 5 columns
            if pred.shape[1] < 5:  # Pad if less than 5 columns
              padding = np.zeros((pred.shape[0], 5-pred.shape[1]))
              pred = np.concatenate((pred, padding), axis=1)

        predicted_dg[subj_idx, 0] = pred

    # Save predictions
    scipy.io.savemat('enhanced_predicted_submission.mat', {'predicted_dg': predicted_dg})
    print("\n✅ Enhanced predictions saved to 'enhanced_predicted_submission.mat'")

## CNN

In [10]:
# ========== CNN Model Definition ==========
class CNNModel(nn.Module):
    def __init__(self, input_dim, output_dim, hidden_dims=[256, 128, 64], kernel_sizes=[5, 3, 3]):
        super().__init__()
        layers = []
        in_channels = 1  # Treat features as channels for 1D conv

        for i, (h_dim, k_size) in enumerate(zip(hidden_dims, kernel_sizes)):
            layers.extend([
                nn.Conv1d(in_channels, h_dim, kernel_size=k_size, padding=k_size//2),
                nn.BatchNorm1d(h_dim),
                nn.ReLU(),
                nn.MaxPool1d(2)
            ])
            in_channels = h_dim

        self.cnn = nn.Sequential(*layers)

        # Calculate flattened size after convolutions
        self.flattened_size = self._get_flattened_size(input_dim, hidden_dims, kernel_sizes)

        self.fc = nn.Sequential(
            nn.Linear(self.flattened_size, 128),
            nn.ReLU(),
            nn.Linear(128, output_dim)
        )

    def _get_flattened_size(self, input_dim, hidden_dims, kernel_sizes):
        # Simulate forward pass to get flattened size
        x = torch.zeros(1, 1, input_dim)  # (batch, channels, length)
        for h_dim, k_size in zip(hidden_dims, kernel_sizes):
            x = nn.Conv1d(x.shape[1], h_dim, k_size, padding=k_size//2)(x)
            x = nn.MaxPool1d(2)(x)
        return x.shape[1] * x.shape[2]

    def forward(self, x):
        # Input shape: (batch, seq_len, features) -> reshape for CNN
        if x.dim() == 3:
            x = x.permute(0, 2, 1)  # (batch, features, seq_len)
        elif x.dim() == 2:
            x = x.unsqueeze(1)  # (batch, 1, features)

        x = self.cnn(x)
        x = x.view(x.size(0), -1)  # Flatten
        return self.fc(x)


In [11]:
# ========== Correlation Loss ==========
class CorrelationLoss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, preds, targets):
        # Input shapes: (batch, features)
        preds = preds - preds.mean(dim=0)
        targets = targets - targets.mean(dim=0)

        cov = (preds * targets).mean(dim=0)
        pred_std = torch.sqrt((preds**2).mean(dim=0))
        target_std = torch.sqrt((targets**2).mean(dim=0))

        corr = cov / (pred_std * target_std + 1e-8)
        return -corr.mean()  # Negative because we want to maximize correlation

# ========== Hybrid Model ==========
class HybridECoGModel:
    def __init__(self, fs=1000, window_len=100, overlap=50, N_wind=3, n_splits=5):
        self.fs = fs
        self.window_len = window_len
        self.overlap = overlap
        self.step = window_len - overlap
        self.N_wind = N_wind
        self.n_splits = n_splits
        self.scaler = StandardScaler()
        self.model = None
        self.criterion = CorrelationLoss()

    def bandpass_filter(self, data, lowcut, highcut, order=4):
        nyq = 0.5 * self.fs
        low = lowcut / nyq
        high = highcut / nyq
        b, a = signal.butter(order, [low, high], btype='band')
        return signal.filtfilt(b, a, data, axis=0)

    def get_features(self, window):
        freq_bands = [(5, 15), (20, 25), (75, 115), (125, 160), (160, 175)]
        n_channels = window.shape[1]
        features = np.zeros((n_channels, 6))
        features[:, 0] = np.mean(window, axis=0)
        for i, (low, high) in enumerate(freq_bands):
            band_filtered = self.bandpass_filter(window, low, high)
            features[:, i + 1] = np.mean(np.abs(band_filtered), axis=0)
        return features

    def get_windowed_feats(self, ecog):
        num_windows = (ecog.shape[0] - self.overlap) // self.step
        feats = []
        for i in range(num_windows):
            start = i * self.step
            end = start + self.window_len
            if end > ecog.shape[0]:
                break
            window = ecog[start:end, :]
            feats.append(self.get_features(window).flatten())
        return np.array(feats)

    def create_R_matrix(self, features):
        num_windows, num_feats = features.shape
        pad = np.tile(features[0], (self.N_wind - 1, 1))
        padded = np.vstack([pad, features])
        R = np.zeros((num_windows, 1 + self.N_wind * num_feats))
        for i in range(num_windows):
            context = padded[i:i + self.N_wind].flatten()
            R[i] = np.concatenate(([1], context))
        return R

    def train(self, ecog, glove):
        # Feature extraction
        feats = self.get_windowed_feats(ecog)
        R = self.create_R_matrix(feats)

        # Downsample glove data
        glove_down = signal.decimate(glove, glove.shape[0] // R.shape[0], axis=0, zero_phase=True)[:R.shape[0]].copy()

        # Smooth glove data
        for i in range(glove_down.shape[1]):
            glove_down[:, i] = gaussian_filter1d(glove_down[:, i], sigma=2.25)

        # Prepare data
        X = R[:, 1:]  # Remove bias term
        y = glove_down

        # Verify output dimensions
        assert y.shape[1] == 5, f"Expected glove data with 5 dimensions, got {y.shape[1]}"

        # Normalize
        X = self.scaler.fit_transform(X)

        # Convert to PyTorch tensors
        X_tensor = torch.FloatTensor(X).to(device)
        y_tensor = torch.FloatTensor(y).to(device)

        # Initialize model
        input_dim = X.shape[1]
        output_dim = y.shape[1]
        self.model = CNNModel(input_dim, output_dim).to(device)

        # Training setup
        optimizer = torch.optim.Adam(self.model.parameters(), lr=1e-3, weight_decay=1e-5)
        scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', patience=5)

        # Create dataset
        dataset = torch.utils.data.TensorDataset(X_tensor, y_tensor)
        train_loader = DataLoader(dataset, batch_size=32, shuffle=True)

        # Training loop
        n_epochs = 100
        best_loss = float('inf')

        for epoch in range(n_epochs):
            self.model.train()
            epoch_loss = 0
            for batch_X, batch_y in train_loader:
                optimizer.zero_grad()
                outputs = self.model(batch_X)
                loss = self.criterion(outputs, batch_y)
                loss.backward()
                optimizer.step()
                epoch_loss += loss.item()

            epoch_loss /= len(train_loader)
            scheduler.step(epoch_loss)

            if epoch_loss < best_loss:
                best_loss = epoch_loss
                torch.save(self.model.state_dict(), 'best_model.pth')

            if (epoch + 1) % 10 == 0:
                print(f'Epoch {epoch+1}/{n_epochs}, Loss: {epoch_loss:.4f}')

    def predict(self, ecog, batch_size=32):
        if self.model is None:
            raise ValueError("Model not trained yet")

        # Load best model
        self.model.load_state_dict(torch.load('best_model.pth'))
        self.model.eval()

        # Feature extraction
        feats = self.get_windowed_feats(ecog)
        R = self.create_R_matrix(feats)
        X = R[:, 1:]
        X = self.scaler.transform(X)

        # Make predictions in batches
        predictions = []
        for i in range(0, len(X), batch_size):
            batch = X[i:i+batch_size]
            X_tensor = torch.FloatTensor(batch).to(device)
            with torch.no_grad():
                batch_pred = self.model(X_tensor)
            predictions.append(batch_pred.cpu().numpy())

        pred = np.concatenate(predictions)
        assert pred.shape[1] == 5, "Model should output 5 dimensions"

        # Pad to required length if needed
        if pred.shape[0] < 147500:
            padding = np.zeros((147500 - pred.shape[0], 5))
            pred = np.vstack([pred, padding])
        else:
            pred = pred[:147500]

        return pred

In [12]:
# ========== Main Execution ==========
if __name__ == "__main__":
    # Load data
    from google.colab import drive
    drive.mount('/content/drive')

    train_data = scipy.io.loadmat('/content/drive/MyDrive/raw_training_data.mat')
    test_data = scipy.io.loadmat('/content/drive/MyDrive/leaderboard_data.mat')
    train_ecogs = train_data['train_ecog']
    train_gloves = train_data['train_dg']
    leaderboard_ecogs = test_data['leaderboard_ecog']

    # Initialize model
    model = HybridECoGModel()

    # Train and predict for each subject
    predicted_dg = np.empty((3, 1), dtype=object)

    for subj_idx in range(3):
        print(f"\n=== Processing Subject {subj_idx + 1} ===")
        ecog_train = train_ecogs[subj_idx].item()
        glove_train = train_gloves[subj_idx].item()
        ecog_test = leaderboard_ecogs[subj_idx].item()

        # Train model
        print("Training model...")
        model.train(ecog_train, glove_train)

        # Make predictions
        print("Making predictions...")
        pred = model.predict(ecog_test, batch_size=32)

        predicted_dg[subj_idx, 0] = pred

    # Save predictions
    scipy.io.savemat('enhanced_predicted_submission.mat', {'predicted_dg': predicted_dg})
    print("\n✅ Enhanced predictions saved to 'enhanced_predicted_submission.mat'")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

=== Processing Subject 1 ===
Training model...
Epoch 10/100, Loss: -0.9466
Epoch 20/100, Loss: -0.9561
Epoch 30/100, Loss: -0.9809
Epoch 40/100, Loss: -0.9842
Epoch 50/100, Loss: -0.9850
Epoch 60/100, Loss: -0.9979
Epoch 70/100, Loss: -0.9992
Epoch 80/100, Loss: -0.9994
Epoch 90/100, Loss: -0.9995
Epoch 100/100, Loss: -0.9996
Making predictions...

=== Processing Subject 2 ===
Training model...
Epoch 10/100, Loss: -0.9270
Epoch 20/100, Loss: -0.9660
Epoch 30/100, Loss: -0.9813
Epoch 40/100, Loss: -0.9926
Epoch 50/100, Loss: -0.9977
Epoch 60/100, Loss: -0.9987
Epoch 70/100, Loss: -0.9990
Epoch 80/100, Loss: -0.9991
Epoch 90/100, Loss: -0.9991
Epoch 100/100, Loss: -0.9991
Making predictions...

=== Processing Subject 3 ===
Training model...
Epoch 10/100, Loss: -0.9611
Epoch 20/100, Loss: -0.9786
Epoch 30/100, Loss: -0.9783
Epoch 40/100, Loss: -0.9851
Epoch 50/